# Create and Test Vega-Lite UC Function (Simple Version)

This is a simplified version that returns hardcoded Vega-Lite specs.
Use this to test the UC Function creation process without needing Foundation Model access.

## Prerequisites
- USAGE + CREATE permissions on your catalog.schema
- That's it! No LLM access needed.

In [0]:
%pip install databricks-sdk unitycatalog-ai typing_extensions>=4.7.0 databricks-connect>=15.1.0 --upgrade
dbutils.library.restartPython()  # Comment out for local execution via dbconnect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.5/219.5 kB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 1: Configure Your Catalog and Schema

In [0]:
# TODO: EDIT THESE VALUES
CATALOG = "your_catalog"  # Change to your catalog name
SCHEMA = "your_schema"  # Change to your schema name

# Verify you have permissions
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")
print(f"✅ Using {CATALOG}.{SCHEMA}")

✅ Using main_david_thomas.default


## Step 2: Define a Simple Vega-Lite Generator

This version returns template specs based on keywords (no LLM needed).

In [0]:
"""
Enhanced Vega-Lite Specification Generator for Databricks Unity Catalog

This function generates Vega-Lite v5 specifications with intelligent multi-scale support.
It automatically detects when metrics have vastly different scales (e.g., USD millions vs Volume thousands)
and uses dual-axis charts or separate subplots for clarity.

Key Features:
- Smart field type detection (time, category, value fields)
- Year-over-year comparison support
- Multi-scale metric visualization (dual-axis, subplots)
- Grouped bar charts for multi-metric analysis
- Line charts with multi-series support
- Scatter plots, pie charts, and more

"""


def generate_vega_lite_spec(
    chart_description: str,
    data_sample: str
) -> str:
    """
    Generate a Vega-Lite visualization specification from a description.
    
    Enhanced version with multi-scale support for comparing metrics with different scales.
    
    Args:
        chart_description (str): Natural language description of the chart.
        data_sample (str): JSON string array of objects.
    
    Returns:
        str: JSON string containing a valid Vega-Lite v5 specification
    """
    import json
    import re
    
    # Validation
    if not chart_description or not chart_description.strip():
        return json.dumps({
            "error": "chart_description cannot be empty",
            "status": "failed"
        })
    
    # Parse data with fallback
    data_values = []
    try:
        if data_sample and data_sample.strip():
            parsed = json.loads(data_sample)
            if isinstance(parsed, list) and len(parsed) > 0:
                data_values = parsed
    except (json.JSONDecodeError, TypeError):
        pass
    
    # Default sample data if none provided
    if not data_values:
        data_values = [
            {"category": "A", "value": 28},
            {"category": "B", "value": 55},
            {"category": "C", "value": 43}
        ]
    
    # Validate data structure
    if not isinstance(data_values, list) or len(data_values) == 0:
        data_values = [{"category": "A", "value": 28}]
    
    # Get field names
    try:
        first_item = data_values[0]
        if not first_item or not isinstance(first_item, dict) or len(first_item.keys()) < 2:
            data_values = [{"category": "A", "value": 28}]
            fields = ["category", "value"]
        else:
            fields = list(first_item.keys())
    except (IndexError, AttributeError, TypeError):
        fields = ["category", "value"]
    
    # Smart field detection - more programmatic and data-driven
    def detect_field_types(fields, data_values):
        """
        Programmatically detect field types based on field names and actual data.
        Returns: (time_field, category_field, value_fields)
        """
        time_field = None
        category_field = None
        value_fields = []
        
        # Keywords for field detection (configurable)
        time_keywords = ['year', 'date', 'time', 'month', 'day', 'quarter', 'week']
        category_keywords = ['name', 'region', 'category', 'type', 'brand', 'segment', 
                           'product', 'customer', 'location', 'country', 'state', 'city']
        value_keywords = ['sales', 'revenue', 'volume', 'amount', 'value', 'price', 
                         'qty', 'quantity', 'usd', 'count', 'total', 'sum', 'avg', 'mean']
        
        for field in fields:
            field_lower = field.lower()
            
            # Get sample value for type checking
            sample_value = None
            is_numeric = False
            is_string = False
            try:
                sample_value = data_values[0][field]
                is_numeric = isinstance(sample_value, (int, float)) and not isinstance(sample_value, bool)
                is_string = isinstance(sample_value, str)
            except:
                pass
            
            # TIME FIELD detection
            if any(keyword in field_lower for keyword in time_keywords):
                if is_numeric or is_string:
                    time_field = field
                    continue
            elif is_string and sample_value and re.match(r'^\d{4}', str(sample_value)):
                if time_field is None:
                    time_field = field
                    continue
            
            # VALUE FIELD detection
            if is_numeric:
                has_value_keyword = any(keyword in field_lower for keyword in value_keywords)
                has_time_keyword = any(keyword in field_lower for keyword in time_keywords)
                has_id_keyword = 'id' in field_lower
                
                if has_value_keyword or (not has_time_keyword and not has_id_keyword):
                    value_fields.append(field)
                    continue
            
            # CATEGORY FIELD detection
            if is_string:
                has_category_keyword = any(keyword in field_lower for keyword in category_keywords)
                
                if has_category_keyword and category_field is None:
                    category_field = field
                elif category_field is None:
                    category_field = field
        
        # Fallback logic
        if not value_fields:
            for field in reversed(fields):
                try:
                    if isinstance(data_values[0][field], (int, float)):
                        value_fields = [field]
                        break
                except:
                    pass
            if not value_fields:
                value_fields = [fields[-1]]
        
        if not category_field and not time_field:
            for field in fields:
                try:
                    if not isinstance(data_values[0][field], (int, float)):
                        category_field = field
                        break
                except:
                    pass
            if not category_field:
                category_field = fields[0]
        
        return time_field, category_field, value_fields
    
    # Helper: Detect if metrics have vastly different scales
    def have_different_scales(value_fields, data_values):
        """
        Check if value fields have significantly different scales (>100x difference).
        Returns: (bool, scale_info)
        """
        if len(value_fields) < 2:
            return False, {}
        
        # Calculate ranges for each metric
        ranges = {}
        for field in value_fields:
            try:
                values = [row[field] for row in data_values if field in row and isinstance(row[field], (int, float))]
                if values:
                    ranges[field] = {
                        'min': min(values),
                        'max': max(values),
                        'range': max(values) - min(values) if max(values) != min(values) else max(values)
                    }
            except:
                pass
        
        if len(ranges) < 2:
            return False, {}
        
        # Compare scales
        field_list = list(ranges.keys())
        max_ratio = 1
        for i in range(len(field_list)):
            for j in range(i + 1, len(field_list)):
                field1, field2 = field_list[i], field_list[j]
                if ranges[field2]['range'] > 0:
                    ratio = ranges[field1]['range'] / ranges[field2]['range']
                    max_ratio = max(max_ratio, ratio, 1/ratio if ratio > 0 else 1)
        
        # If any ratio > 100, they have different scales
        different_scales = max_ratio > 100
        
        return different_scales, ranges
    
    time_field, category_field, value_fields = detect_field_types(fields, data_values)
    
    # Determine primary value field
    primary_value_field = value_fields[0] if value_fields else fields[-1]
    
    description_lower = chart_description.lower()
    
    # Detect multi-scale scenario
    different_scales, scale_info = have_different_scales(value_fields, data_values)
    
    # Base configuration
    base_spec = {
        "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
        "description": chart_description,
        "data": {"values": data_values},
        "width": 500,
        "height": 300
    }
    
    # Chart type logic
    if any(word in description_lower for word in ["line", "trend", "time series"]):
        # LINE CHART - Time series
        x_field = time_field or category_field or fields[0]
        y_field = primary_value_field
        
        # Determine if multi-series (has both time and category)
        color_field = None
        if time_field and category_field:
            x_field = time_field
            color_field = category_field
        
        # Determine x-axis type
        x_type = "ordinal"
        if time_field == x_field:
            try:
                sample_value = data_values[0][x_field]
                if isinstance(sample_value, (int, float)):
                    x_type = "ordinal"
                elif isinstance(sample_value, str):
                    if re.match(r'^\d{4}-\d{2}-\d{2}', str(sample_value)):
                        x_type = "temporal"
                    else:
                        x_type = "ordinal"
            except:
                x_type = "ordinal"
        
        encoding = {
            "x": {
                "field": x_field,
                "type": x_type,
                "title": x_field.replace("_", " ").title()
            },
            "y": {
                "field": y_field,
                "type": "quantitative",
                "title": y_field.replace("_", " ").title()
            }
        }
        
        if color_field:
            encoding["color"] = {
                "field": color_field,
                "type": "nominal",
                "title": color_field.replace("_", " ").title()
            }
        
        return json.dumps({
            **base_spec,
            "mark": {
                "type": "line",
                "point": True,
                "tooltip": True
            },
            "encoding": encoding,
            "config": {
                "view": {"stroke": None}
            }
        })
    
    elif any(word in description_lower for word in ["bar", "column", "histogram"]):
        # BAR CHART
        x_field = category_field or time_field or fields[0]
        
        # Check if this is a year-over-year comparison
        is_year_comparison = time_field and (
            'compare' in description_lower or
            'vs' in description_lower or
            'versus' in description_lower or
            description_lower.count('year') >= 2
        )
        
        # Check if user wants multi-metric chart
        wants_multi_metric = (
            len(value_fields) > 1 and 
            any(keyword in description_lower for keyword in 
                ["grouped", "two bars", "both", "multiple", "including", "compare", "two separate", "side by side", "separate charts"])
        )
        
        # MULTI-METRIC HANDLING (prioritize explicit user requests)
        if wants_multi_metric:
            # PRIORITY 1: Check if user explicitly wants side-by-side or separate charts
            if any(keyword in description_lower for keyword in ["side by side", "separate charts", "two separate"]):
                # Determine orientation
                use_horizontal = "side by side" in description_lower
                
                charts = []
                for value_field in value_fields:
                    chart_spec = {
                        "title": {
                            "text": value_field.replace("_", " ").title(),
                            "fontSize": 14
                        },
                        "width": 400 if use_horizontal else 500,
                        "height": 300 if use_horizontal else 200,
                        "mark": {"type": "bar", "tooltip": True},
                        "encoding": {
                            "x": {
                                "field": x_field,
                                "type": "nominal" if category_field == x_field else "ordinal",
                                "axis": {"labelAngle": -45},
                                "title": x_field.replace("_", " ").title()
                            },
                            "y": {
                                "field": value_field,
                                "type": "quantitative",
                                "title": None
                            }
                        }
                    }
                    
                    # Add year color encoding if year comparison
                    if is_year_comparison:
                        chart_spec["encoding"]["color"] = {
                            "field": time_field,
                            "type": "ordinal",
                            "title": time_field.replace("_", " ").title(),
                            "scale": {"range": ["#4c78a8", "#f58518"]}  # blue and orange
                        }
                        chart_spec["encoding"]["xOffset"] = {"field": time_field}
                    
                    charts.append(chart_spec)
                
                concat_key = "hconcat" if use_horizontal else "vconcat"
                return json.dumps({
                    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
                    "description": chart_description,
                    "data": {"values": data_values},
                    concat_key: charts,
                    "resolve": {
                        "scale": {"y": "independent"}
                    },
                    "config": {
                        "view": {"stroke": None}
                    }
                })
            
            # PRIORITY 2: DUAL-AXIS CHART (for 2 metrics with different scales, no explicit layout request)
            elif different_scales and len(value_fields) == 2:
                return json.dumps({
                    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
                    "description": chart_description,
                    "data": {"values": data_values},
                    "width": 500,
                    "height": 300,
                    "layer": [
                        {
                            "mark": {"type": "bar", "opacity": 0.7, "color": "#4c78a8", "tooltip": True},
                            "encoding": {
                                "x": {
                                    "field": x_field,
                                    "type": "nominal" if category_field == x_field else "ordinal",
                                    "axis": {"labelAngle": -45},
                                    "title": x_field.replace("_", " ").title()
                                },
                                "y": {
                                    "field": value_fields[0],
                                    "type": "quantitative",
                                    "title": value_fields[0].replace("_", " ").title(),
                                    "axis": {"titleColor": "#4c78a8"}
                                }
                            }
                        },
                        {
                            "mark": {
                                "type": "line",
                                "color": "#e45756",
                                "point": {"filled": True, "size": 80},
                                "strokeWidth": 3,
                                "tooltip": True
                            },
                            "encoding": {
                                "x": {
                                    "field": x_field,
                                    "type": "nominal" if category_field == x_field else "ordinal"
                                },
                                "y": {
                                    "field": value_fields[1],
                                    "type": "quantitative",
                                    "title": value_fields[1].replace("_", " ").title(),
                                    "axis": {"titleColor": "#e45756", "orient": "right"}
                                }
                            }
                        }
                    ],
                    "resolve": {
                        "scale": {"y": "independent"}
                    },
                    "config": {
                        "view": {"stroke": None}
                    }
                })
            
            # PRIORITY 3: VERTICAL SUBPLOTS (for 3+ metrics or different scales)
            elif different_scales or len(value_fields) > 2:
                charts = []
                for value_field in value_fields:
                    chart_spec = {
                        "title": {
                            "text": value_field.replace("_", " ").title(),
                            "fontSize": 14
                        },
                        "width": 500,
                        "height": 200,
                        "mark": {"type": "bar", "tooltip": True},
                        "encoding": {
                            "x": {
                                "field": x_field,
                                "type": "nominal" if category_field == x_field else "ordinal",
                                "axis": {"labelAngle": -45},
                                "title": x_field.replace("_", " ").title()
                            },
                            "y": {
                                "field": value_field,
                                "type": "quantitative",
                                "title": None
                            }
                        }
                    }
                    
                    # Add year color encoding if year comparison
                    if is_year_comparison:
                        chart_spec["encoding"]["color"] = {
                            "field": time_field,
                            "type": "ordinal",
                            "title": time_field.replace("_", " ").title()
                        }
                        chart_spec["encoding"]["xOffset"] = {"field": time_field}
                    
                    charts.append(chart_spec)
                
                return json.dumps({
                    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
                    "description": chart_description,
                    "data": {"values": data_values},
                    "vconcat": charts,
                    "resolve": {
                        "scale": {"y": "independent"}
                    },
                    "config": {
                        "view": {"stroke": None}
                    }
                })
            
            # PRIORITY 4: STANDARD GROUPED BARS (same scale)
            else:
                transformed_data = []
                for row in data_values:
                    for value_field in value_fields:
                        transformed_data.append({
                            x_field: row[x_field],
                            "metric": value_field.replace("_", " ").title(),
                            "value": row[value_field]
                        })
                
                return json.dumps({
                    **base_spec,
                    "data": {"values": transformed_data},
                    "mark": {
                        "type": "bar",
                        "tooltip": True
                    },
                    "encoding": {
                        "x": {
                            "field": x_field,
                            "type": "nominal" if category_field == x_field else "ordinal",
                            "axis": {"labelAngle": -45}
                        },
                        "y": {
                            "field": "value",
                            "type": "quantitative",
                            "title": "Value"
                        },
                        "color": {
                            "field": "metric",
                            "type": "nominal",
                            "title": "Metric"
                        },
                        "xOffset": {
                            "field": "metric"
                        }
                    },
                    "config": {
                        "view": {"stroke": None}
                    }
                })
        
        # YEAR COMPARISON (single metric)
        elif is_year_comparison:
            primary_metric = value_fields[0] if value_fields else fields[-1]
            
            return json.dumps({
                **base_spec,
                "mark": {
                    "type": "bar",
                    "tooltip": True
                },
                "encoding": {
                    "x": {
                        "field": x_field,
                        "type": "nominal" if category_field == x_field else "ordinal",
                        "axis": {"labelAngle": -45}
                    },
                    "y": {
                        "field": primary_metric,
                        "type": "quantitative",
                        "title": primary_metric.replace("_", " ").title()
                    },
                    "color": {
                        "field": time_field,
                        "type": "ordinal",
                        "title": time_field.replace("_", " ").title()
                    },
                    "xOffset": {
                        "field": time_field
                    }
                },
                "config": {
                    "view": {"stroke": None}
                }
            })
        
        # SINGLE METRIC BAR CHART
        y_field = primary_value_field
        
        return json.dumps({
            **base_spec,
            "mark": {
                "type": "bar",
                "tooltip": True
            },
            "encoding": {
                "x": {
                    "field": x_field,
                    "type": "nominal" if category_field == x_field else "ordinal",
                    "axis": {"labelAngle": -45}
                },
                "y": {
                    "field": y_field,
                    "type": "quantitative"
                }
            },
            "config": {
                "view": {"stroke": None}
            }
        })
    
    elif any(word in description_lower for word in ["scatter", "point"]):
        # SCATTER PLOT
        x_field = fields[0] if len(fields) >= 2 else "x"
        y_field = fields[1] if len(fields) >= 2 else "y"
        
        return json.dumps({
            **base_spec,
            "mark": {
                "type": "point",
                "tooltip": True
            },
            "encoding": {
                "x": {"field": x_field, "type": "quantitative"},
                "y": {"field": y_field, "type": "quantitative"}
            },
            "config": {
                "view": {"stroke": None}
            }
        })
    
    elif any(word in description_lower for word in ["pie", "donut"]):
        # PIE CHART
        category = category_field or fields[0]
        value = primary_value_field
        
        return json.dumps({
            **base_spec,
            "mark": {"type": "arc", "tooltip": True},
            "encoding": {
                "theta": {"field": value, "type": "quantitative"},
                "color": {"field": category, "type": "nominal"}
            }
        })
    
    else:
        # DEFAULT: BAR CHART
        x_field = category_field or fields[0]
        y_field = primary_value_field
        
        return json.dumps({
            **base_spec,
            "mark": {
                "type": "bar",
                "tooltip": True
            },
            "encoding": {
                "x": {
                    "field": x_field,
                    "type": "nominal",
                    "axis": {"labelAngle": -45}
                },
                "y": {
                    "field": y_field,
                    "type": "quantitative"
                }
            },
            "config": {
                "view": {"stroke": None}
            }
        })


## Step 3: Register as UC Function

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from databricks.sdk import WorkspaceClient

# Initialize clients
client = DatabricksFunctionClient()
w = WorkspaceClient()  # For getting workspace URL

full_function_name = f"{CATALOG}.{SCHEMA}.generate_vega_lite_spec"

print(f"Creating UC Function: {full_function_name}")

# Create the function using the high-level API
function_info = client.create_python_function(
    func=generate_vega_lite_spec,
    catalog=CATALOG,
    schema=SCHEMA,
    replace=True  # Overwrites existing function if it exists
)

print(f"✅ UC Function created successfully!")
print(f"Function name: {function_info.full_name}")
print(f"\n📍 MCP URL: {w.config.host}/api/2.0/mcp/functions/{CATALOG}/{SCHEMA}")

/local_disk0/.ephemeral_nfs/envs/pythonEnv-31673932-c449-4265-adb5-1fefb9392461/lib/python3.10/site-packages/databricks/connect/session.py:451: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)


Creating UC Function: main_david_thomas.default.generate_vega_lite_spec
✅ UC Function created successfully!
Function name: main_david_thomas.default.generate_vega_lite_spec

📍 MCP URL: https://adb-361426925668745.5.azuredatabricks.net/api/2.0/mcp/functions/main_david_thomas/default


## Step 4: Test the UC Function

In [0]:
import json

# Reuse the DatabricksFunctionClient for testing
# client = DatabricksFunctionClient()  # Already initialized above

print("Test 1: Bar chart with default data")
print("-" * 60)
result_1 = client.execute_function(
    full_function_name,
    {"chart_description": "bar chart of sales", "data_sample": ""}  # Empty string for default data
)
print(f"✅ Result:\n{json.dumps(json.loads(result_1.value), indent=2)}")
print()

print("Test 2: Line chart with custom data")
print("-" * 60)
sample_data = json.dumps([
    {"month": "Jan", "revenue": 10000},
    {"month": "Feb", "revenue": 15000},
    {"month": "Mar", "revenue": 13000},
    {"month": "Apr", "revenue": 18000}
])
result_2 = client.execute_function(
    full_function_name,
    {"chart_description": "line chart showing revenue trend", "data_sample": sample_data}
)
print(f"✅ Result:\n{json.dumps(json.loads(result_2.value), indent=2)}")

Test 1: Bar chart with default data
------------------------------------------------------------
✅ Result:
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "bar chart of sales",
  "data": {
    "values": [
      {
        "category": "A",
        "value": 28
      },
      {
        "category": "B",
        "value": 55
      },
      {
        "category": "C",
        "value": 43
      }
    ]
  },
  "width": 500,
  "height": 300,
  "mark": {
    "type": "bar",
    "tooltip": true
  },
  "encoding": {
    "x": {
      "field": "category",
      "type": "nominal",
      "axis": {
        "labelAngle": -45
      }
    },
    "y": {
      "field": "value",
      "type": "quantitative"
    }
  },
  "config": {
    "view": {
      "stroke": null
    }
  }
}

Test 2: Line chart with custom data
------------------------------------------------------------
✅ Result:
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "line chart 

## Step 5: Visualize the Result

In [0]:
# Visualize the line chart
vega_spec = json.loads(result_2.value)

# displayHTML only works in Databricks notebooks, not local execution
try:
    displayHTML(f"""
<!DOCTYPE html>
<html>
<head>
  <script src="https://cdn.jsdelivr.net/npm/vega@5"></script>
  <script src="https://cdn.jsdelivr.net/npm/vega-lite@5"></script>
  <script src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
  <style>
    body {{ background: #f5f5f5; padding: 20px; }}
    #vis {{ background: white; padding: 20px; border-radius: 8px; }}
  </style>
</head>
<body>
  <h2>Generated Vega-Lite Visualization</h2>
  <div id="vis"></div>
  <script>
    vegaEmbed('#vis', {json.dumps(vega_spec)}, {{theme: 'latimes'}});
  </script>
</body>
</html>
""")
except NameError:
    print("Note: displayHTML only works in Databricks notebooks, not local execution")
    print(f"Vega spec generated:\n{json.dumps(vega_spec, indent=2)}")

<!DOCTYPE html>
 
 
 
 
 
 
 
 
 Generated Vega-Lite Visualization

## Step 6: Test in Serverless Mode (Production)

In [0]:
print("Testing scatter plot...")
print("-" * 60)
result_prod = client.execute_function(
    full_function_name,
    {
        "chart_description": "scatter plot",
        "data_sample": json.dumps([
            {"x": 1, "y": 2},
            {"x": 2, "y": 4},
            {"x": 3, "y": 6},
            {"x": 4, "y": 8}
        ])
    }
)
print(f"✅ Execution successful!")
print(f"Result: {result_prod.value[:100]}...")

Testing scatter plot...
------------------------------------------------------------
✅ Execution successful!
Result: {"$schema": "https://vega.github.io/schema/vega-lite/v5.json", "description": "scatter plot", "data"...


## ✅ Success! Your UC Function is Ready

### What You Created:
- ✅ UC Function: `{CATALOG}.{SCHEMA}.generate_vega_lite_spec`
- ✅ Tested in local mode (fast)
- ✅ Tested in serverless mode (production)
- ✅ Visualized the output

### MCP Access:
```
{workspace_url}/api/2.0/mcp/functions/{CATALOG}/{SCHEMA}
```

### Next Steps:
1. Configure your agent to use this MCP endpoint
2. The agent will auto-discover `generate_vega_lite_spec`
3. When users ask for visualizations, the agent calls your function
4. Your app renders the returned Vega-Lite spec

### Grant Permissions:
```sql
GRANT EXECUTE ON FUNCTION {full_function_name} TO `<principal>`;
```